In [1]:
import mlflow
from mlflow.tracking import MlflowClient
import optuna
import re
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [5]:
df_DST = pd.read_csv("../../data/raw/DST.csv")
df_K = pd.read_csv("../../data/raw/K.csv")
df_QB = pd.read_csv("../../data/raw/QB.csv")
df_RB = pd.read_csv("../../data/raw/RB.csv")
df_TE = pd.read_csv("../../data/raw/TE.csv")
df_WR = pd.read_csv("../../data/raw/WR.csv")

df_DST['Position'] = 'DST'
df_K['Position'] = 'K'
df_QB['Position'] = 'QB'
df_RB['Position'] = 'RB'
df_TE['Position'] = 'TE'
df_WR['Position'] = 'WR'

df = pd.concat([df_DST, df_K, df_QB, df_RB, df_TE, df_WR], ignore_index=True)

print(f"Total de filas: {len(df)}")
print(f"\nDistribución por posición:")
print(df['Position'].value_counts())
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nColumnas:")
print(df.columns.tolist())
print(f"\nInfo del DataFrame:")
print(df.info())

Total de filas: 983

Distribución por posición:
Position
WR     339
RB     222
TE     201
QB     123
K       64
DST     34
Name: count, dtype: int64

Primeras filas:
   Rank                     Player  SACK   INT    FR    FF  DEF TD  SFTY  \
0   1.0       Houston Texans (HOU)  33.0  12.0   7.0   8.0     2.0   0.0   
1   2.0     Los Angeles Rams (LAR)  31.0  12.0   7.0  10.0     1.0   0.0   
2   3.0     Seattle Seahawks (SEA)  36.0   9.0   4.0   3.0     2.0   0.0   
3   4.0     Cleveland Browns (CLE)  42.0   9.0   6.0  10.0     2.0   0.0   
4   5.0  Pittsburgh Steelers (PIT)  34.0   9.0  11.0  13.0     3.0   0.0   

   SPC TD     G  ...  TD  SACKS ATT.1 YDS.1  TD.1  FL  20+  TGT  REC  Y/R  
0     0.0  11.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
1     0.0  11.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
2     2.0  11.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN  
3     0.0  11.0  ... NaN    NaN   NaN   NaN   NaN NaN  NaN  NaN  NaN  NaN

In [6]:
# ============================================================
# PREP: Codificación de 'Position' + Selección de features
# ============================================================

# ---------- 1) Definir target ----------
TARGET = "FPTS"

# ---------- 2) Quitar columnas que NO deben ser features ----------
# - Identificadores y texto
id_like = ["Player", "Team"]  # agrega otras si las tienes (e.g., 'PlayerId')
# - Fugas de información (derivadas del target o rankings)
leak_like_patterns = [
    r"^FPTS\/G$",      # puntos por juego (deriva del target)
    r"rank",           # cualquier 'rank' o variantes
    r"tier",           # tiers si existieran
]
# Compilar regex para filtrar
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df.columns if leak_regex.search(str(c))])

# ---------- 3) Seleccionar columnas numéricas y categóricas ----------
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]

# Asegurar que Position esté como categórica
cat_cols = ["Position"]

# ---------- 4) Imputación + OneHot para 'Position' ----------
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

# ---------- 5) Ajustar transformador y generar X, y ----------
X = preprocessor.fit_transform(df)
y = df[TARGET].values

# ---------- 6) Obtener nombres de features transformadas ----------
ohe_feature_names = preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols)
feature_names = np.r_[num_cols, ohe_feature_names]

print(f"Total features numéricas: {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(ohe_feature_names)}")
print(f"Total de features finales: {len(feature_names)}")

# Vista rápida de las primeras 25 columnas transformadas
print("\nEjemplo de nombres de features resultantes:")
print(feature_names[:25])

Total features numéricas: 32
Total features categóricas (one-hot): 6
Total de features finales: 38

Ejemplo de nombres de features resultantes:
['SACK' 'INT' 'FR' 'FF' 'DEF TD' 'SFTY' 'SPC TD' 'G' 'FG' 'FGA' 'PCT' 'LG'
 '1-19' '20-29' '30-39' '40-49' '50+' 'XPT' 'XPA' 'CMP' 'ATT' 'Y/A' 'TD'
 'SACKS' 'ATT.1']


In [7]:
# ------------------ 0) Configuración ------------------
TARGET = "FPTS"

# ------------------ 1) Asegurar tipos numéricos y target limpio ------------------
df2 = df.copy()

# Forzar TARGET a numérico
df2[TARGET] = pd.to_numeric(df2[TARGET], errors="coerce")

# Quitar filas con FPTS NaN/inf
mask = np.isfinite(df2[TARGET])
df2 = df2.loc[mask].reset_index(drop=True)

print(f"Filas después de limpiar {TARGET}: {len(df2)}")

# ------------------ 2) Definir columnas a eliminar (no-features) ------------------
# Identificadores y texto que no deben entrar como features
id_like = ["Player", "Team"]  # si faltan, se manejan luego para imprimir
# Fugas de información: cualquier cosa derivada del target o rankings
leak_like_patterns = [
    r"^FPTS\/G$",   # puntos por juego (deriva del target)
    r"\brank\b",    # rank, Rank, RANK
    r"\btier\b",    # tier, Tier, TIER
]
leak_regex = re.compile("|".join(leak_like_patterns), flags=re.IGNORECASE)

drop_cols = set(id_like + [TARGET])
drop_cols.update([c for c in df2.columns if leak_regex.search(str(c))])

# ------------------ 3) Columnas numéricas y categóricas ------------------
# Asegurar que Position exista
if "Position" not in df2.columns:
    raise ValueError("No se encontró la columna 'Position' en el DataFrame.")

num_cols = [c for c in df2.select_dtypes(include=[np.number]).columns
            if c not in drop_cols and c != TARGET]
cat_cols = ["Position"]

print(f"Total features numéricas (detectadas): {len(num_cols)}")
print(f"Total features categóricas (one-hot): {len(cat_cols)} → {cat_cols}")


# ------------------ 4) Preprocesamiento ------------------
numeric_transformer = SimpleImputer(strategy="constant", fill_value=0)
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ],
    remainder="drop"
)

Filas después de limpiar FPTS: 971
Total features numéricas (detectadas): 32
Total features categóricas (one-hot): 1 → ['Position']


In [ ]:
df.to_csv('../../data/processed/df_draft_players.csv', index=False, encoding='utf-8')